In [1]:
import os
from PIL import Image
folder_path="/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/toothbrush/train/good/"
path_os=os.listdir(folder_path)
img_path=[folder_path+i for i in path_os]
PIL_image=[Image.open(i).convert("RGB") for i in img_path]


In [2]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [3]:
from transformers import AutoImageProcessor, AutoModel
model_name = "facebook/dino-vitb8"
processor = AutoImageProcessor.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)
encoder.eval()


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vitb8 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): ViTOutput(
          (dense

In [4]:
import torch

normal_embeddings = []

for img in PIL_image:
    x = transform(img)

    with torch.no_grad():
        inputs = processor(images=x, return_tensors="pt", do_rescale=False)
        outputs = encoder(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :]   # CLS token
    embedding = embedding.squeeze(0)                 # shape: (768,)
    normal_embeddings.append(embedding)


In [5]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Convert list of tensors → 2D numpy array
normal_embeddings_np = torch.stack(normal_embeddings).numpy()

knn = NearestNeighbors(n_neighbors=5)
knn.fit(normal_embeddings_np)

,n_neighbors,5
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'minkowski'
,p,2
,metric_params,None
,n_jobs,None


In [6]:
import joblib

joblib.dump(knn, "knn_toothbrush_good_model.pkl")
print("Saved KNN model!")

Saved KNN model!


In [7]:
# --- Load a Good-test image ---
test_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/toothbrush/test/good/000.png"
img = Image.open(test_path).convert("RGB")

# --- Preprocess ---
x = transform(img)

# --- Get test embedding ---
with torch.no_grad():
    inputs = processor(images=x, return_tensors="pt", do_rescale=False)
    outputs = encoder(**inputs)

test_embed = outputs.last_hidden_state[:, 0, :].squeeze(0)  # (768,)

# --- Convert to numpy ---
test_embed_np = test_embed.numpy().reshape(1, -1)

# --- Compute distance using k-NN ---
dist, _ = knn.kneighbors(test_embed_np)

anomaly_score = dist.mean()
print("Anomaly Score:", anomaly_score)

Anomaly Score: 11.582470321655274


In [8]:
# --- Load a defective-test image ---
test_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/toothbrush/test/defective/000.png"
img = Image.open(test_path).convert("RGB")

# --- Preprocess ---
x = transform(img)

# --- Get test embedding ---
with torch.no_grad():
    inputs = processor(images=x, return_tensors="pt", do_rescale=False)
    outputs = encoder(**inputs)

test_embed = outputs.last_hidden_state[:, 0, :].squeeze(0)  # (768,)

# --- Convert to numpy ---
test_embed_np = test_embed.numpy().reshape(1, -1)

# --- Compute distance using k-NN ---
dist, _ = knn.kneighbors(test_embed_np)

anomaly_score = dist.mean()
print("Anomaly Score:", anomaly_score)

Anomaly Score: 23.184053039550783


In [9]:
import os
import numpy as np
import torch
from PIL import Image
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_fscore_support
import pandas as pd

# allowed image extensions
valid_ext = [".png", ".jpg", ".jpeg", ".bmp"]

# paths
test_root = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/toothbrush/test/"
good_dir = os.path.join(test_root, "good")

# anomalies: any folder except 'good'
anomaly_dirs = [
    os.path.join(test_root, d)
    for d in os.listdir(test_root)
    if d != "good" and os.path.isdir(os.path.join(test_root, d))
]

# ---------- PRINT PATHS ----------
print("Good dir:", good_dir)
print("Anomaly dirs:", anomaly_dirs)

print("\nFiles in good:", os.listdir(good_dir))
for ad in anomaly_dirs:
    print("Files in", ad, ":", os.listdir(ad))


# helper to get embedding (returns numpy 1x768)
def get_embed_from_pil(img_pil):
    x = transform(img_pil)  # transform defined earlier
    with torch.no_grad():
        inputs = processor(images=x, return_tensors="pt", do_rescale=False)
        outputs = encoder(**inputs)
    emb = outputs.last_hidden_state[:, 0, :].squeeze(0)  # (768,)
    return emb.cpu().numpy().reshape(1, -1)


# evaluate images
scores = []
labels = []
paths = []

count_good = 0
count_anomaly = 0


# -------------------------
# GOOD images (label = 0)
# -------------------------
for fn in sorted(os.listdir(good_dir)):
    if not any(fn.lower().endswith(ext) for ext in valid_ext):
        continue

    p = os.path.join(good_dir, fn)

    try:
        img = Image.open(p).convert("RGB")
    except:
        continue

    emb_np = get_embed_from_pil(img)
    dist, _ = knn.kneighbors(emb_np)
    score = dist.mean()

    scores.append(float(score))
    labels.append(0)
    paths.append(p)

    count_good += 1


# -------------------------
# ANOMALY images (label = 1)
# -------------------------
for ad in anomaly_dirs:
    for fn in sorted(os.listdir(ad)):
        if not any(fn.lower().endswith(ext) for ext in valid_ext):
            continue

        p = os.path.join(ad, fn)

        try:
            img = Image.open(p).convert("RGB")
        except:
            continue

        emb_np = get_embed_from_pil(img)
        dist, _ = knn.kneighbors(emb_np)
        score = dist.mean()

        scores.append(float(score))
        labels.append(1)
        paths.append(p)

        count_anomaly += 1


# ---------- PRINT COUNTS ----------
print("\nGood images processed:", count_good)
print("Anomaly images processed:", count_anomaly)
print("Total processed:", count_good + count_anomaly)


# convert to numpy
scores = np.array(scores)
labels = np.array(labels)

# -------------------------
# Compute performance metrics
# -------------------------
roc = roc_auc_score(labels, scores)
pr = average_precision_score(labels, scores)

# find best threshold by maximizing F1
thresholds = np.linspace(scores.min(), scores.max(), 200)
best_f1 = 0.0
best_thresh = thresholds[0]
best_prec = best_rec = 0.0

for t in thresholds:
    preds = (scores >= t).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary', zero_division=0
    )
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t
        best_prec = prec
        best_rec = rec

# ---------- PRINT METRICS ----------
print("\nAUC-ROC:", round(roc, 4))
print("AUC-PR :", round(pr, 4))
print(f"Best F1: {best_f1:.4f} at threshold {best_thresh:.4f}")
print(f"Precision: {best_prec:.4f}, Recall: {best_rec:.4f}")


# save results
df = pd.DataFrame({"path": paths, "label": labels, "score": scores})
df.to_csv("toothbrush_test_scores_knn.csv", index=False)
print("\nSaved toothbrush_test_scores_knn.csv")


Good dir: /Users/ggharish13/Data Science/Capstone Project/Final Project/Images/toothbrush/test/good
Anomaly dirs: ['/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/toothbrush/test/defective']

Files in good: ['002.png', '003.png', '001.png', '000.png', '010.png', '004.png', '005.png', '011.png', '007.png', '006.png', '008.png', '009.png']
Files in /Users/ggharish13/Data Science/Capstone Project/Final Project/Images/toothbrush/test/defective : ['002.png', '016.png', '017.png', '003.png', '029.png', '015.png', '001.png', '000.png', '014.png', '028.png', '010.png', '004.png', '005.png', '011.png', '007.png', '013.png', '012.png', '006.png', '023.png', '022.png', '008.png', '020.png', '021.png', '009.png', '025.png', '019.png', '018.png', '024.png', '026.png', '027.png']

Good images processed: 12
Anomaly images processed: 30
Total processed: 42

AUC-ROC: 0.9611
AUC-PR : 0.9854
Best F1: 0.9508 at threshold 14.2715
Precision: 0.9355, Recall: 0.9667

Saved toothbrush_tes

In [10]:
import torch
from PIL import Image
import numpy as np

best_threshold = 16   # from evaluation

def predict_image(path):
    # load image
    img = Image.open(path).convert("RGB")
    
    # get embedding
    emb_np = get_embed_from_pil(img)   # function defined earlier
    
    # kNN distance
    dist, _ = knn.kneighbors(emb_np)
    score = dist.mean()

    # classification
    label = "DEFECTIVE" if score >= best_threshold else "GOOD"

    return score, label


In [11]:
img_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/toothbrush/test/good/007.png"

score, label = predict_image(img_path)
print("Score:", score)
print("Prediction:", label)


Score: 15.995209503173829
Prediction: GOOD
